In [ ]:
from __future__ import print_function
import torchvision
import torchvision.transforms as transforms
import os
import time
import sys

sys.path.insert(0, './SpikingNN')
sys.path.insert(0, './n_caltech101')

from SpikingNN.spiking_model import*
from n_caltech101.neuro_dataset import*
import numpy as np

import train as yolo

import glob
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def bounding_box(class_name, class_index, offset, dest):
    w = 240
    h = 176

    for file in glob.glob("N-Caltech101/Caltech101_annotations/" + class_name + "/*"):    
        print(file)

        # BB in: x,y,w,h (point in center)
        # YOLO BB: [[batch_index, class, x, y, w, h]] normalized to 0-1

        f = open(file, "r")
        a = np.fromfile(f, dtype=np.int16)

        xywh = torch.tensor([[0, class_index, a[2]/w, a[3]/h, (a[6] - a[2])/w, (a[7] - a[3])/h]])
        print(xywh)

        torch.save(xywh.to(device), dest + str(int(file[-8:-4]) + offset) + ".pt")
        
dest = "N-Caltech101/tensor_annotations/"
if not os.path.exists(dest):
    os.makedirs(dest)

class_name = "airplanes"  # Name of the folder for the class
offset = 0 # Offset label of data so that it is stored continuously. 
class_index = 0
bounding_box(class_name, class_index, offset, dest)

class_name = "Motorbikes"
offset = 800 
class_index = 1
bounding_box(class_name, class_index, offset, dest)

In [ ]:
def time_rate_encoding(class_name, offset, dest):
    width = 240
    height = 176
    
    time_step = int(300e3 / 20) # 15,000 micro seconds for 20 frames

    for file in glob.glob("N-Caltech101/Caltech101/" + class_name + "/*"):    
        print(file)
        events = ev.read_dataset(file)
        sampling_stop = time_step

        images = torch.zeros([0, 1, 2, height, width])
        image = torch.zeros([1, 1, 2, height, width])
        for i, e in enumerate(events.data):
            if e[3] >= sampling_stop:  # Event frame finished
                if sampling_stop >= 300e3:  # Data point finished
                    break

                images = torch.cat((images, image))
                image = torch.zeros([1, 1, 2, height, width])
                sampling_stop += time_step

            image[0][0][int(e[2])][e[1]][e[0]] += 1

        images = torch.cat((images, image))
        torch.save(images.to(device), dest + str(int(file[-8:-4]) + offset) + ".pt")
        print(images.shape)
        
dest = "N-Caltech101/tensor_data/"
if not os.path.exists(dest):
    os.makedirs(dest)
    
class_name = "airplanes"
offset = 0
time_rate_encoding(class_name, offset, dest)

class_name = "Motorbikes"
offset = 800 
time_rate_encoding(class_name, offset, dest)